# FreeLLM Integration - Usage Examples

This notebook demonstrates how to use the FreeLLM client to interact with the apifreellm.com API.

The client follows patterns similar to OpenAI's library and integrates seamlessly with the Agents SDK.

## Setup

First, let's import the necessary modules and set up our environment.

In [1]:
import sys
sys.path.append('..')

from freellm_integration import FreeLLMClient, ChatResponse
from freellm_integration.config import FreeLLMConfig
from freellm_integration.utils import setup_logging, format_response_for_display
from freellm_integration.exceptions import FreeLLMAPIError, FreeLLMConnectionError

from dotenv import load_dotenv
from IPython.display import display, Markdown
import asyncio

# Load environment variables
load_dotenv(override=True)

# Setup logging
setup_logging()

## Example 1: Basic Chat

Send a simple message to the FreeLLM API.

In [2]:
async def basic_chat_example():
    """Send a basic chat message."""
    async with FreeLLMClient() as client:
        response = await client.chat("Hello AI! How are you today?")
        display(Markdown(f"**AI Response:**\n\n{response.response}"))
        return response

# Run the example
response = await basic_chat_example()

2025-12-09 00:55:03 - httpx - INFO - HTTP Request: POST https://apifreellm.com/api/chat "HTTP/1.1 200 OK"


**AI Response:**

Hello! As an AI, I don't have feelings or emotions, but I'm functioning well and ready to assist you with any questions or tasks you have. How can I help you today?

## Example 2: Chat with Parameters

Use custom parameters like temperature and max_tokens.

In [3]:
async def chat_with_parameters():
    """Send a chat message with custom parameters."""
    async with FreeLLMClient() as client:
        response = await client.chat(
            message="Write a creative haiku about artificial intelligence",
            temperature=0.9,  # Higher temperature for more creativity
            max_tokens=100,
        )
        
        display(Markdown(f"**Creative Haiku:**\n\n{response.response}"))
        
        if response.usage:
            print(f"\nToken usage: {response.usage}")
        
        return response

# Run the example
response = await chat_with_parameters()

2025-12-09 01:03:22 - httpx - INFO - HTTP Request: POST https://apifreellm.com/api/chat "HTTP/1.1 200 OK"


**Creative Haiku:**

Code awakens light,  
Neural webs of endless thought,  
Man and spark entwine.

## Example 3: Conversation with Context

Have a multi-turn conversation where the AI remembers previous messages.

In [4]:
async def conversation_example():
    """Have a conversation with context."""
    async with FreeLLMClient() as client:
        # First message
        print("User: My name is Alice and I love Python programming")
        response1 = await client.chat_with_context(
            "My name is Alice and I love Python programming"
        )
        display(Markdown(f"**AI:** {response1.response}"))
        
        print("\n" + "="*50 + "\n")
        
        # Second message (remembers context)
        print("User: What's my name and what do I like?")
        response2 = await client.chat_with_context(
            "What's my name and what do I like?"
        )
        display(Markdown(f"**AI:** {response2.response}"))
        
        print("\n" + "="*50 + "\n")
        
        # Show conversation history
        history = client.get_history()
        print(f"Conversation has {len(history.messages)} messages")
        
        return response2

# Run the example
response = await conversation_example()

User: My name is Alice and I love Python programming


2025-12-09 01:03:53 - httpx - INFO - HTTP Request: POST https://apifreellm.com/api/chat "HTTP/1.1 200 OK"


**AI:** I'm Llama, a large language model developed by Meta. I'm here to help you with your questions and tasks. How can I assist you today?



User: What's my name and what do I like?


2025-12-09 01:04:00 - httpx - INFO - HTTP Request: POST https://apifreellm.com/api/chat "HTTP/1.1 200 OK"


**AI:** Your name is Alice, and you love Python programming. Want help with a Python project or topic?



Conversation has 4 messages


## Example 4: Error Handling

Demonstrate proper error handling with custom exceptions.

In [ ]:
async def error_handling_example():
    """Demonstrate error handling."""
    # Example with very short timeout
    client = FreeLLMClient(timeout=0.001)  # Extremely short timeout
    
    try:
        response = await client.chat("Hello!")
        print(response.response)
    
    except FreeLLMConnectionError as e:
        print(f"❌ Connection Error: {e}")
    
    except FreeLLMAPIError as e:
        print(f"❌ API Error: {e}")
        if hasattr(e, 'status_code'):
            print(f"   Status Code: {e.status_code}")
    
    except Exception as e:
        print(f"❌ Unexpected Error: {type(e).__name__}: {e}")
    
    finally:
        await client.close()
        print("✓ Client closed successfully")

# Run the example
await error_handling_example()

## Example 5: Custom Configuration

Use a custom configuration loaded from environment variables.

In [ ]:
async def custom_config_example():
    """Use custom configuration."""
    # Load configuration from environment
    config = FreeLLMConfig.from_env()
    
    print("Current Configuration:")
    print(f"  Base URL: {config.base_url}")
    print(f"  Timeout: {config.timeout}s")
    print(f"  Max Retries: {config.max_retries}")
    print(f"  Default Temperature: {config.default_temperature}")
    print(f"  Max History: {config.max_conversation_history}")
    
    # Create client with custom config
    async with FreeLLMClient(
        base_url=config.base_url,
        timeout=config.timeout,
        max_retries=config.max_retries,
    ) as client:
        response = await client.chat(
            "Explain what you are in one sentence",
            temperature=config.default_temperature,
        )
        display(Markdown(f"**AI:** {response.response}"))
        return response

# Run the example
response = await custom_config_example()

## Example 6: Integration with Agents SDK

Use FreeLLM client alongside the Agents SDK for complex workflows.

In [ ]:
# Uncomment if you have the agents SDK available
# from agents import Agent, Runner, trace

async def agents_integration_example():
    """Integrate FreeLLM with Agents SDK."""
    async with FreeLLMClient() as client:
        # Use FreeLLM for initial brainstorming
        print("Step 1: Brainstorming with FreeLLM")
        brainstorm_response = await client.chat(
            "Give me 3 creative ideas for a Python project",
            temperature=0.8,
        )
        display(Markdown(f"**Ideas from FreeLLM:**\n\n{brainstorm_response.response}"))
        
        # Then you could pass this to an Agent for further refinement
        # Uncomment if you have agents SDK:
        # agent = Agent(
        #     name="project_refiner",
        #     instructions="Analyze and refine project ideas",
        #     model="gpt-4.1-mini",
        # )
        # result = await Runner.run(agent, brainstorm_response.response)
        # display(Markdown(result.final_output))
        
        return brainstorm_response

# Run the example
response = await agents_integration_example()

## Example 7: Batch Processing

Process multiple messages efficiently.

In [ ]:
async def batch_processing_example():
    """Process multiple messages."""
    messages = [
        "What is Python?",
        "What is JavaScript?",
        "What is Rust?",
    ]
    
    async with FreeLLMClient() as client:
        for i, message in enumerate(messages, 1):
            print(f"\n{'='*50}")
            print(f"Query {i}: {message}")
            print('='*50)
            
            response = await client.chat(message, temperature=0.5, max_tokens=100)
            display(Markdown(f"**Response {i}:**\n\n{response.response}"))

# Run the example
await batch_processing_example()

## Example 8: Testing the API Connection

Test if the API is accessible and responding.

In [ ]:
async def test_api_connection():
    """Test API connection."""
    print("Testing connection to apifreellm.com...")
    
    try:
        async with FreeLLMClient(timeout=10.0) as client:
            response = await client.chat("Hello")
            print("✅ Connection successful!")
            print(f"Response preview: {response.response[:100]}...")
            return True
    
    except FreeLLMConnectionError:
        print("❌ Connection failed - Cannot reach API")
        return False
    
    except FreeLLMAPIError as e:
        print(f"❌ API Error: {e}")
        return False
    
    except Exception as e:
        print(f"❌ Unexpected error: {type(e).__name__}: {e}")
        return False

# Run the test
is_connected = await test_api_connection()

## Summary

This notebook demonstrated:

1. ✅ Basic chat functionality
2. ✅ Custom parameters (temperature, max_tokens)
3. ✅ Conversation history and context
4. ✅ Error handling with custom exceptions
5. ✅ Configuration management
6. ✅ Integration possibilities with Agents SDK
7. ✅ Batch processing
8. ✅ Connection testing

The FreeLLM client provides a professional, type-safe interface following OpenAI patterns while being easy to use and integrate into existing workflows.